# IMDB movies dataset analysis using SQLite and Pandas

In [2]:
import sqlite3
import pandas as pd
import numpy as np

In [4]:
db = sqlite3.connect('IMDB.sqlite')

## Top votes

In [5]:
df = pd.read_sql_query('Select movie_id,title, rating, (cvotesmale + cvotesfemale) as totalvote from IMDB',db)
max = df['totalvote'].max()
df = df[df['totalvote'] == max]

print(df['Title'].values[0], df['Rating'].values[0])

Inception (2010) 8.8


## Longest runtime

In [8]:
df = pd.read_sql_query('Select movie_id,title, rating, runtime from IMDB',db)

def finding_time(input_runtime):
    return input_runtime.strip().split(" ")[0]

df['Runtime'] = df['Runtime'].apply(finding_time)

df = df[df['Runtime'] != '']    # '' blanks cannot be converted into integer so it need to be filtered out

df.Runtime = df.Runtime.astype(int)
res = df.sort_values( by = 'Runtime', ascending = False)

print(res['Title'].values[0], res['Runtime'].values[0])

The Wolf of Wall Street (2013) 180


## Most profitable movie

In [10]:

df = pd.read_sql_query('Select title, (domestic+worldwide-(IMDB.budget)) as net_profit from IMDB inner join earning on IMDB.movie_id = earning.movie_id',db)
res = df.sort_values(by = 'net_profit', ascending = False)

print(res['Title'].values[0])

Star Wars: The Force Awakens (2015)


## Least budget movie

In [31]:
df = pd.read_sql_query('Select title,budget, (domestic+worldwide-(IMDB.budget)) as net_profit from IMDB inner join earning on IMDB.movie_id = earning.movie_id',db)
df['Budget'].replace('', np.nan, inplace = True)
df.dropna(subset = ['Budget'], inplace = True)
df['Budget'].astype(float)

df = df[df['Budget'] == min(df['Budget'])]

df = df[df['net_profit'] == df['net_profit'].max()]
print(df['Title'].values[0])

Short Term 12 (2013)


##  Year wise top movies

In [32]:
df = pd.read_sql_query('select IMDB.movie_id, title, domestic, worldwide from IMDB inner join earning on IMDB.movie_id = earning.movie_id',db)
def year(title):
    return int(title.strip().split(' ')[-1][-5:-1])

df['year'] = df['Title'].apply(year)

df['Total'] = df['Domestic'] + df['Worldwide']

group = df.groupby('year')['Total'].max()

dic = {}
for i in range(len(df['Title'])):
    if df['Total'][i] in group.values:
        key = df['year'][i]
        value = df['Title'][i]
        dic[key] = value
        
for i in sorted(dic):
    print(i,dic[i])

2010 Toy Story 3 (2010)
2011 Rise of the Planet of the Apes (2011)
2012 The Avengers (2012)
2013 The Hunger Games: Catching Fire (2013)
2014 Guardians of the Galaxy (2014)
2015 Star Wars: The Force Awakens (2015)
2016 Rogue One (2016)


##  Budget percentage genre wise

In [33]:
df = pd.read_sql_query('select IMDB.movie_id, genre, title, budget from IMDB inner join genre on IMDB.movie_id = genre.movie_id',db)
df

df['Budget'].replace('', 0 , inplace = True)
df['Budget'] = df['Budget'].astype(float)
df["genre"].replace('', np.nan , inplace = True)
df.dropna(subset = ['genre'], inplace = True)
group = df.groupby('genre')['Budget'].sum()
for i in range(len(group)):
    pct = (group.values[i]/sum(group.values))*100
    print(group.index[i], format(pct, '.2f'))

Action 17.98
Adventure 25.10
Animation 7.66
Biography 2.59
Comedy 7.22
Crime 1.77
Drama 10.85
Family 1.91
Fantasy 4.54
History 0.72
Horror 0.03
Music 0.15
Musical 0.25
Mystery 1.95
Romance 1.12
Sci-Fi 10.77
Sport 0.46
Thriller 3.99
War 0.36
Western 0.57


##  Earning percentage genre wise

In [34]:
df = pd.read_sql_query('select earning.movie_id, genre, domestic, worldwide from earning inner join genre on earning.movie_id = genre.movie_id',db)


df['Total'] = df['Domestic'] + df['Worldwide']
df['genre'].replace('', np.nan, inplace = True)
df.dropna(subset = ['genre'] , inplace = True)
group = df.groupby('genre')['Total'].sum()

for i in range(len(group)):
    pct = (group.values[i]/sum(group.values))*100
    print(group.index[i], format(pct, '.2f'))


Action 17.91
Adventure 23.57
Animation 6.77
Biography 3.42
Comedy 7.69
Crime 1.55
Drama 11.96
Family 1.05
Fantasy 5.15
History 1.04
Horror 0.01
Music 0.43
Musical 0.38
Mystery 2.01
Romance 1.45
Sci-Fi 9.95
Sport 0.32
Thriller 4.42
War 0.25
Western 0.65


##  Top rated movie in Sci-fi Mystery

In [36]:
df = pd.read_sql_query('Select title, rating, genre from IMDB inner join genre on IMDB.movie_id = genre.movie_id',db)

x =  ((df['genre']== 'Mystery')|(df['genre'] == 'Sci-Fi')) & (df['Rating']>= 8)

df = df[x]
print(df['Title'].values[0])

Arrival (2016)
